### Default to Iteration 1: Above-the-Line / Below-the-Line (ATL/BTL) Testing

In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
import statistics as stat
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from src.data_models.SmartsCsvDataModel import SmartsCsvDataModel
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.UPMhelper import UPMalerts

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Step 0) Lookback Days {20,30,40}

In [2]:
lookback_prod = unSMART(pd.read_excel('~/Documents/vmfldr/official_test.xlsx'))
lookback_01 = unSMART(pd.read_excel('~/Documents/vmfldr/TEST_LB01_18DEC.xlsx'))
lookback_20 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_TEST0.xlsx'))
lookback_30 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_TEST1.xlsx'))
lookback_40 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_TEST2.xlsx'))

-> type fixed, kept 354 rows, 13 columns
-> type fixed, kept 19630 rows, 13 columns
-> type fixed, kept 36263 rows, 13 columns
-> type fixed, kept 36884 rows, 13 columns
-> type fixed, kept 38127 rows, 13 columns


In [3]:
lookback_prod['ShortText3'] = lookback_prod['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
lookback_01['ShortText3'] = lookback_01['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
lookback_20['ShortText3'] = lookback_20['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
lookback_30['ShortText3'] = lookback_30['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
lookback_40['ShortText3'] = lookback_40['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')

In [4]:
lookback_prodm = lookback_prod.loc[lookback_prod['Datetime'].dt.month == 12]
lookback_01m = lookback_01.loc[lookback_01['Datetime'].dt.month == 12]
lookback_20m = lookback_20.loc[lookback_20['Datetime'].dt.month == 12]
lookback_30m = lookback_30.loc[lookback_30['Datetime'].dt.month == 12]
lookback_40m = lookback_40.loc[lookback_40['Datetime'].dt.month == 12]

print(lookback_01m.shape)
print(lookback_20m.shape)
print(lookback_30m.shape)
print(lookback_40m.shape)

(18974, 14)
(18407, 14)
(18566, 14)
(18606, 14)


In [5]:
byCode_01 = pd.DataFrame(lookback_01m.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'LB_01'})
byCode_20 = pd.DataFrame(lookback_20m.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'LB_20'})
byCode_30 = pd.DataFrame(lookback_30m.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'LB_30'})
byCode_40 = pd.DataFrame(lookback_40m.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'LB_40'})

In [6]:
lookback_impact = pd.concat([byCode_01,byCode_20,byCode_30,byCode_40], axis=1)
lookback_impact['LB_01to20'] = round(lookback_impact['LB_20']/lookback_impact['LB_01'] - 1,2) * 100
lookback_impact['LB_20to30'] = round(lookback_impact['LB_30']/lookback_impact['LB_20'] - 1,2) * 100
lookback_impact['LB_30to40'] = round(lookback_impact['LB_40']/lookback_impact['LB_30'] - 1,2) * 100
lookback_impact

LB_01  LB_20  LB_30  LB_40  \
AlertCode ShortText3                                                     
1001      UNUSUAL PRICE MOVEMENT INTRA-DAY    107     92     98     94   
4009      PAINTING THE TAPE                    36     34     39     34   
4012      PRICE DRIVER                         32     32     32     32   
4022      BAIT AND SWITCH                   15218  14668  14816  14865   
4032      MULTI ORDER SPOOFING                433    433    433    433   
          MULTI ORDER SPOOFING EOD            415    415    415    415   
4041      WASH SALE A-B-A                      28     28     28     28   
4042      WASH SALE A-B-A (MONEY PASS)       2702   2702   2702   2702   
4045      COLLUSION(MONEY PASS)                 3      3      3      3   

                                            LB_01to20  LB_20to30  LB_30to40  
AlertCode ShortText3                                                         
1001      UNUSUAL PRICE MOVEMENT INTRA-DAY      -14.0        7.0       -4.0  
4009      PAINTING THE TAPE                      -6.0       15.0      -13.0  
4012      PRICE DRIVER                            0.0        0.0        0.0  
4022      BAIT AND SWITCH                        -4.0        1.0        0.0  
4032      MULTI ORDER SPOOFING                    0.0        0.0        0.0  
          MULTI ORDER SPOOFING EOD                0.0        0.0        0.0  
4041      WASH SALE A-B-A                         0.0        0.0        0.0  
4042      WASH SALE A-B-A (MONEY PASS)            0.0        0.0        0.0  
4045      COLLUSION(MONEY PASS)                   0.0        0.0        0.0

### Values -1001

In [7]:
regextext1 = '\D+(Price Change.+)Bid.+'
regextext2 = 'Price Change\s(?:in the last )?(\d+\D+|trade to trade)\sis\D+([\d.]+)\D+([\d.]+)\D+[\d.]+\D+[\d.]+\D+([\d.]+)\D+([\d.]+)%\)\s+'

alerts1001 = lookback_30m.loc[lookback_30m['AlertCode'] == 1001,['AlertCode','LongText','InstrumentName']]
alerts1001['LongText2'] = alerts1001['LongText'].str.extract(regextext1,expand=True)
alerts1001_rematch = alerts1001['LongText2'].str.extractall(regextext2).reset_index()
alerts1001_rematch.columns = ['index','index2','range','valdiff','valperc','bmdiff','bmperc']
alerts1001_rematch['symbol'] = alerts1001_rematch.reset_index()['index'].map(alerts1001['InstrumentName'].to_dict())
alerts1001_rematch.groupby('range')['valperc'].count()

range
1 hour            55
10 minutes        74
trade to trade    64
Name: valperc, dtype: int64

In [8]:
alerts1001.shape

(98, 4)

In [9]:
regextext1 = '\D+(Price Change.+)Bid.+'
regextext2 = 'Price Change\s(?:in the last )?(\d+\D+|trade to trade)\sis\D+([\d.]+)\D+([\d.]+)\D+[\d.]+\D+[\d.]+\D+([\d.]+)\D+([\d.]+)%\)\s+'

alerts1001 = lookback_prod.loc[lookback_prod['AlertCode'] == 1001,['AlertCode','LongText','InstrumentName']]
alerts1001['LongText2'] = alerts1001['LongText'].str.extract(regextext1,expand=True)
alerts1001_rematch = alerts1001['LongText2'].str.extractall(regextext2).reset_index()
alerts1001_rematch.columns = ['index','index2','range','valdiff','valperc','bmdiff','bmperc']
alerts1001_rematch['symbol'] = alerts1001_rematch.reset_index()['index'].map(alerts1001['InstrumentName'].to_dict())
alerts1001_rematch.groupby('range')['valperc'].count()

range
1 hour        16
10 minutes    31
5 minutes     32
Name: valperc, dtype: int64

In [10]:
alerts1001.shape

(32, 4)

### Values -2011

In [11]:
alerts2011 = lookback_prodm.loc[lookback_prodm['AlertCode'] == 2011,['AlertCode','LongText','InstrumentName']]
regextext3 = '[^:]*\:*\s*([A-Z]{6})\D+([\d,.]+)\D+(\d+\D+)\.\D+([\d,.]+)\s*'
alerts2011_rematch = alerts2011['LongText'].str.extractall(regextext3).reset_index()
alerts2011_rematch.columns = ['index','index2','symbol','value','range','bm']
alerts2011_rematch.groupby('range')['value'].count()

range
1 hour        6
10 minutes    6
5 minutes     8
Name: value, dtype: int64

In [12]:
alerts2011_rematch

,index,index2,symbol,value,range,bm
0,229,0,ZECUSD,3471.2887,10 minutes,3436
1,252,0,BCHUSD,622.424029,10 minutes,617
2,256,0,LTCBTC,7556.79547,5 minutes,7509
3,258,0,LTCUSD,10655.04075,5 minutes,10599
4,273,0,BTCUSD,2177.833357,1 hour,2177
5,275,0,ZECBCH,47.09,5 minutes,47
6,275,1,ZECBCH,47.09,10 minutes,47
7,276,0,BCHBTC,308.352646,1 hour,307
8,277,0,BCHUSD,1195.606105,1 hour,1190
9,292,0,BCHUSD,1190.405242,1 hour,1190


### Values -4012

In [13]:
# alerts4012_rematch['dir'].hist(bins=30); plt.show()
# alerts4012_rematch['vol'].hist(bins=30)

In [14]:
# alerts4012_rematch['dir'].hist(bins=30); plt.show()
# alerts4012_rematch['tick'].hist(bins=30)

In [37]:
lookback = lookback_prodm
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
# for i in alerts4012['LongText']: print(i,'\n')
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
alerts4012_rematch
alerts4012_rematch[['dir','tick','vol']].to_csv('~/Desktop/tuned_pdinvestigation.csv')

In [36]:
# lookback_prod.groupby('AlertCode').count()

In [31]:
alerts4012

,AlertCode,LongText,InstrumentName
248,4012,PRICE DRIVER DOWN (BROKER): Broker 44498 has d...,BCHUSD
249,4012,PRICE DRIVER UP (BROKER): Broker 44498 has inc...,ZECBTC
257,4012,PRICE DRIVER DOWN (BROKER): Broker 772544 has ...,LTCBTC
267,4012,PRICE DRIVER DOWN (BROKER): Broker 303565 has ...,ZECETH
283,4012,PRICE DRIVER UP (BROKER): Broker 411770 has in...,ETHBTC
284,4012,PRICE DRIVER DOWN (BROKER): Broker 44498 has d...,ETHBTC
285,4012,PRICE DRIVER DOWN (BROKER): Broker 44498 has d...,BCHUSD
294,4012,PRICE DRIVER UP (BROKER): Broker 411770 has in...,ETHBTC
295,4012,PRICE DRIVER DOWN (BROKER): Broker 44498 has d...,BCHUSD
302,4012,PRICE DRIVER DOWN (BROKER): Broker 44498 has d...,BCHUSD


In [21]:
lookback = lookback_01m
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
# for i in alerts4012['LongText']: print(i,'\n')
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
alerts4012_rematch
alerts4012_rematch[['dir','tick','vol']].to_csv('~/Desktop/tuned01.csv')

In [22]:
lookback = lookback_20m
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
# for i in alerts4012['LongText']: print(i,'\n')
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
alerts4012_rematch
alerts4012_rematch[['dir','tick','vol']].to_csv('~/Desktop/tuned20.csv')

In [23]:
lookback = lookback_30m
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
# for i in alerts4012['LongText']: print(i,'\n')
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3)
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
alerts4012_rematch.shape
alerts4012_rematch[['dir','tick','vol']].to_csv('~/Desktop/tuned30.csv')

In [24]:
lookback = lookback_40m
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
# for i in alerts4012['LongText']: print(i,'\n')
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
alerts4012_rematch
alerts4012_rematch[['dir','tick','vol']].to_csv('~/Desktop/tuned40.csv')